# `hdp_py` Documentation

## HDP

A class implementing two Gibbs sampler formulations for the Hierarchical Dirichlet Process.

**Constructor parameters:**
+ `gamma`, `alpha0`: scaling parameters > 0 for base measures $H$ and $G_0$
+ `f`: string representing distribution of data; $h$ is chosen to be conjugate.  Options are `'poisson'`, `'multinomial'`, `'categorical'`, or `'categorical_fast'`.
+ `hypers`: tuple of hyperparameter values specific to $f$/$h$ scheme chosen.  For the Poisson model, this is a pair of positive real numbers $(\alpha, \beta)$.  For all others, this is a pair ($L$, ($L$,) numpy array) for some integer $L > 2$.

**Public attributes:**
+ `cfr_samples`: `(iters x N x 2)` matrix of (t, k) values for each data point i; exists only after `gibbs_cfr()` has been called
+ `direct_samples`: `(iters x N)` matrix of k values for each data point i; exists only after `gibbs_direct()` has been called
+ `beta_samples`: `(iters x Kmax+1)` matrix of beta values after each iteration; exists only after `gibbs_direct()` has been called

### gibbs_cfr

Method which implements the Chinese Franchise restaurant formulation of the HDP.  Returns the object.
+ `x`: an `(N, L)` numpy array in which each row represents a single observation.  The value of $L$ is specific to the model for $f$ chosen in the constructor.
+ `j`: a parallel `(N,)` numpy array encoding the group each observation in `x` belongs to.  These should be encoded as non-negative integers, and each observation should belong to exactly one group.
+ `iters`: the number of samples to draw.  This excludes the initial iteration but does not exclude any burn-in.
+ `Tmax`: maximum number of clusters ($t$ values) within any single group.  By default this is the size of the largest group given in `j`.
+ `Kmax`: maximum number of atoms ($k$ values) across the whole dataset.  This is $\min(100, N)$ by default.

### gibbs_direct

Method which implements the direct sampler formulation of the HDP.  This is the faster option and has more optimization features.  This shares all the same arguments as `gibbs_cfr()` except `Tmax` and with the additional options:
+ `resume`: boolean; an option to continue sampling from the previous call, rather than having to start over and use burn-in iterations.
+ `verbose`: boolean; prints out $\mathbf{\beta}$ values after each iteration to visualize clustering 

## get_data

A module containing some helper functions to access pre-formatted data ready for use in the `HDP` class.

+ `get_nematode(max_docs = None, min_word_count = 1, LDA = false)`: accesses the [nematode abstract data]('https://raw.githubusercontent.com/tdhopper/topic-modeling-datasets/master/data/raw/Nematode%20biology%20abstracts/cgcbib.txt') used by the original paper.  Removes stop words and encodes each word appearing at least `min_word_count` times across the first `max_docs` abstracts as a categorical variable drawn from the distribution of all words in the corpus vocabulary.  Returns a dataframe `X` (with vocab given by the column names) and the proper `j` factors.  Note that `X` must be cast to a numpy array before being passed into an `HDP` object.  If `LDA` is True, it instead returns a nested list of individual words from the component documents, suitable for running LDA in `gensim`.
+ `get_reuters(max_docs = None, min_word_count = 1, LDA = false)`: does the same as the above for manually selected Reuters data.  This comes pre-installed in the `data/` directory of the package.
+ `get_test_data(N, L, Jmax)`: simulates a categorical dataset of the given size, returns the same as above.
+ `get_simulated_pop_data()`: returns the data matrix `X`, factor vector `j`, and known latent cluster information `z` for a pre-simulated ant dataset.

### LDA Functions

Some miscellaneous functions for performing a comparative LDA analysis on the included databases .
+ `LDA_preprocessing(data, n_documents, test_size, min_word_count)`: this function takes in data formatted by any of the get_{topic} data functions with LDA = True called.
    
    `n_documents`: the number of documents to select from data. 
    
    `test_size`: the proportion of n_documents that should be held out for testing
    
    `min_word_count`: the minimum number of times a word should appear to be kept in vocabulary
  
    This function returns id2word and corpus for LDA training and testing
    
+ `LDA(id2word, corpus, n_topics)`: wrapper for the `LdaModel` class in the `gensim` package
+ `perplexity(model, test_corpus, test)`: takes a trained LDA model from above and calculates the perplexity.
+ `plt_perplexity(perplexity, min_topics, max_topics)`: this function plots the perplexity given perplexity array returned by above function.  First row of perplexity array is the perplexity values  Second row of perplexity array is the corresponding number of topics used for LDA training